In [ ]:
import numpy as np
import tensorflow as tf
import os

In [ ]:
#Importing training data from drive

data_path='/content/drive/MyDrive/datasets'

In [ ]:
image_size=224
batch_size=128

datagen=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2

)
# Split the data for training and testing

train_generator=datagen.flow_from_directory(
    data_path,
    target_size=(image_size,image_size),
    batch_size=batch_size,
    subset='training')

# Generation of training data from all the samples

val_generator=datagen.flow_from_directory(
    data_path,
    target_size=(image_size,image_size),
    batch_size=batch_size,
    subset='validation'
)

# Genreration of the testing data from all the samples

Found 3597 images belonging to 7 classes.
Found 897 images belonging to 7 classes.


In [ ]:
print(train_generator.class_indices)

labels = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('labels.txt','w') as f:
  f.write(labels)

#Creating the text file with the classes which we have included in the model

{'10 rupees-samples': 0, '100 rupees-samples': 1, '20 rupees-samples': 2, '200 rupees-samples': 3, '2000 rupees-samples': 4, '50 rupees-samples': 5, '500 rupees-samples': 6}


In [ ]:
img_shape=(image_size,image_size,3) #Input image shape

base_model=tf.keras.applications.MobileNetV2(input_shape=img_shape,include_top=False,weights='imagenet')

9420800/9406464 [==============================] - 0s 0us/step


In [ ]:
base_model.trainable=False

In [ ]:
model=tf.keras.Sequential([
    base_model,
    tf.keras.layers.Conv2D(32,3,activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(7,activation='softmax')
])

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
epochs=10

history = model.fit(train_generator,
                    epochs=epochs,
                    validation_data=val_generator)

Epoch 1/10
29/29 [==============================] - 1102s 38s/step - loss: 1.1971 - accuracy: 0.6444 - val_loss: 0.5175 - val_accuracy: 0.8450
Epoch 2/10
29/29 [==============================] - 130s 4s/step - loss: 0.1857 - accuracy: 0.9552 - val_loss: 0.2645 - val_accuracy: 0.9052
Epoch 3/10
29/29 [==============================] - 130s 4s/step - loss: 0.0519 - accuracy: 0.9939 - val_loss: 0.1990 - val_accuracy: 0.9320
Epoch 4/10
29/29 [==============================] - 129s 5s/step - loss: 0.0232 - accuracy: 0.9989 - val_loss: 0.1868 - val_accuracy: 0.9242
Epoch 5/10
29/29 [==============================] - 129s 4s/step - loss: 0.0135 - accuracy: 0.9997 - val_loss: 0.1725 - val_accuracy: 0.9275
Epoch 6/10
29/29 [==============================] - 127s 4s/step - loss: 0.0097 - accuracy: 0.9994 - val_loss: 0.1737 - val_accuracy: 0.9231
Epoch 7/10
29/29 [==============================] - 130s 5s/step - loss: 0.0075 - accuracy: 0.9994 - val_loss: 0.1471 - val_accuracy: 0.9476
Epoch 8/10


In [ ]:
saved_model_dir=''
tf.saved_model.save(model,saved_model_dir)

converter=tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model=converter.convert()

with open('model.tflite','wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: assets


INFO:tensorflow:Assets written to: assets


INFO:tensorflow:Assets written to: /tmp/tmp39sc69yz/assets


INFO:tensorflow:Assets written to: /tmp/tmp39sc69yz/assets


In [ ]:
from google.colab import files

files.download('model.tflite')
files.download('labels.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>